### Imports

In [28]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import time

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation

Using Theano backend.


### Get Data

#### Labels

In [3]:
one_hot = pd.read_csv('../combined_labels/one_hot_labels.csv')
one_hot.head()

,B,C,E,G,H,I,S,T
0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0


In [4]:
one_hot.shape

(268675, 8)

In [36]:
cs = one_hot['C'].values
os = np.ones(len(cs), dtype=int)

print(cs[:10])
print(os[:10])

[1 1 1 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1]


In [45]:
# get indices of 'C' labels, to use as hold-out set
ind = [i for i, l in enumerate(cs) if l==1]

In [47]:
one_hot.drop(ind, inplace=True)

In [49]:
one_hot.shape

(215217, 8)

#### Verbose

In [5]:
matrix = pd.read_csv('../combined_verbose_dssp_csv/(19)_combined_verbose.csv')
matrix.head()

,--9,--8,--7,--6,--5,--4,--3,--2,--1,-0,...,z0,z1,z2,z3,z4,z5,z6,z7,z8,z9
0,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
matrix.drop(ind, inplace=True)

In [6]:
matrix.shape

(268675, 912)

#### Tensor

In [7]:
tensor = np.load('../combined_scaled_tensor_dssp_npy/(19)_combined_scaled.npy')

In [8]:
tensor[0][:5]

array([[ 6.8556546 ,  6.8556546 ,  6.8556546 ,  6.8556546 ,  6.8556546 ,
         6.8556546 ,  6.8556546 ,  6.8556546 ,  6.8556546 , -0.14586499,
        -0.14586499, -0.14586499, -0.14586499, -0.14586499, -0.14586499,
        -0.14586499, -0.14586499, -0.14586499, -0.14586499],
       [-0.14586499, -0.14586499, -0.14586499, -0.14586499, -0.14586499,
        -0.14586499, -0.14586499, -0.14586499, -0.14586499, -0.14586499,
        -0.14586499, -0.14586499, -0.14586499,  6.8556546 , -0.14586499,
        -0.14586499, -0.14586499, -0.14586499, -0.14586499],
       [-0.14586499, -0.14586499, -0.14586499, -0.14586499, -0.14586499,
        -0.14586499, -0.14586499, -0.14586499, -0.14586499, -0.14586499,
        -0.14586499, -0.14586499, -0.14586499, -0.14586499, -0.14586499,
        -0.14586499, -0.14586499, -0.14586499, -0.14586499],
       [-0.14586499, -0.14586499, -0.14586499, -0.14586499, -0.14586499,
        -0.14586499, -0.14586499, -0.14586499, -0.14586499, -0.14586499,
        -0.145

In [9]:
tensor.shape

(268675, 48, 19)

### Splitting Data

In [20]:
X_train, X_test, y_train, y_test = train_test_split(matrix.values, one_hot.values)

In [21]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(201506, 912) (67169, 912) (201506, 8) (67169, 8)


### Simple Two Layer

In [29]:
# instantiate the NN
model = Sequential([
    Dense(input_dim=912, output_dim=500),
    Activation('relu'),
    Dense(input_dim=500, output_dim=8),
    Activation('softmax'),
])

In [30]:
# add the relevant training info
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
fit_model = model.fit(X_train, y_train, nb_epoch=3, batch_size=32, validation_data=(X_test, y_test))
time.sleep(0.1) # fixes I/O error from Keras (https://github.com/fchollet/keras/issues/2110)

Train on 201506 samples, validate on 67169 samples
Epoch 1/3
201506/201506 [==============================] - 183s - loss: 1.2134 - acc: 0.5744 - val_loss: 1.0028 - val_acc: 0.6666
Epoch 2/3
201506/201506 [==============================] - 234s - loss: 0.8029 - acc: 0.7431 - val_loss: 0.8079 - val_acc: 0.7491
Epoch 3/3
 16640/201506 [=>............................] - ETA: 218s - loss: 0.5940 - acc: 0.8169

ValueError: I/O operation on closed file